In [ ]:
import sqlalchemy as sqla
from sqlalchemy import create_engine
import traceback 
import glob
import os 
import pprint
from pprint import pprint
import json
import requests
import time
from IPython.display import display

In [1]:
url = "dbbikes.cvvkn6jkfbdf.eu-west-1.rds.amazonaws.com"
port = "3306"
db = "dbbikes"
user = "SEGroup25"
password = "SEGroup25?"

In [2]:
engine = create_engine(f"mysql+mysqldb://{user}:{password}@{url}:{port}/{db}", echo=True)

NameError: name 'create_engine' is not defined

In [ ]:
for res in engine.execute("SHOW VARIABLES"):
    print(res)

In [ ]:

engine.execute("DROP TABLE IF EXISTS `dbbikes`.`stations`")

sql = """
CREATE TABLE `dbbikes`.`stations` (
  `number` INT NOT NULL,
  `contract_name` VARCHAR(45) NULL,
  `stat_name` VARCHAR(45) NULL,
  `address` VARCHAR(45) NULL,
  `position_lat` REAL NULL,
  `position_long` REAL NULL,
  `banking` TINYINT NULL,
  `bonus` TINYINT NULL,
  `bikestands` VARCHAR(45) NULL,
  `stat_status` VARCHAR(45) NULL,
  PRIMARY KEY (`number`));
"""
engine.execute(sql)



In [ ]:

engine.execute("DROP TABLE IF EXISTS `weather`.`weather`")


sql = """
CREATE TABLE `weather`.`weather` (
  `time` REAL NOT NULL,
  `cord_long` REAL  NULL,
  `cord_lat` REAL NULL,
  `weather_main` VARCHAR(45) NULL,
  `main_temp` REAL NULL,
  `main_humidity` REAL NULL,
  `wind_speed` REAL NULL,
  `wind_degree` REAL NULL,
  `sys_sunset` INT NULL,
  `sys_sunrise` INT NULL
  )
"""
engine.execute(sql)

In [ ]:
sql = """
CREATE TABLE IF NOT EXISTS availability (
available_bike_stands INT,
available_bikes INT,
last_update INT,
number INT)
"""

try:
    res = engine.execute(sql)
    print(res.fetchall())
except Exception as e:
    print(e)

In [3]:
import requests
import traceback
import datetime
import time 

In [4]:
APIKEY = "5ac1b590c7e338d66c34fa0c5610e94c99a3b45c"
NAME = "Dublin"
STATIONS = "https://api.jcdecaux.com/vls/v1/stations"

In [5]:

def write_to_file(text):
    now = datetime.datetime.now()
    with open(f"data/bikes_{now}".replace(" ", "_"), "w") as f:
        f.write(text)
    

            
def stations_to_db(text):
    stations = json.loads(text)
    print(type(stations),len(stations))
    print(text)

    for station in stations:
        print(station)
        vals = (
            station.get("number"),
            station.get("contract_name"),
            station.get("name"),
            station.get("address"),
            station.get("position").get("lat"),
            station.get("position").get("lng"),
            station.get("banking"),
            station.get("bonus"),
            station.get("bike_stands"),
            station.get("status")
        )
        print("\n\n\nVALS:{}\n\n\n".format(vals))
        engine.execute("insert into stations values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)


def availability_to_db(text):
    availability =json.loads(text)
    print(type(availability), len(availability))
    for available in availability:
        print(available)
        vals = (
        (available.get("available_bike_stands")),
        (available.get("available_bikes")),
        available.get("last_update"),
        (available.get("number"))
        )
        engine.execute("insert into availability values(%s, %s, %s, %s)", vals)

    return

    



def main():
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(STATIONS , params = {"apiKey": APIKEY, "contract": NAME})
            print(r,now)
            #write_to_file(r.text)
            stations_to_db(r.text)
            availability_to_db(r.text)
            time.sleep(5 * 60)
        except:
            print(traceback.format_exc())
            if engine is None:
                return

if __name__ == "__main__":
    main()  

<Response [200]> 2023-03-30 10:01:32.246031
Traceback (most recent call last):
  File "/var/folders/7r/5dq81jms6sddg6kh57c8ypwm0000gn/T/ipykernel_56905/1381663942.py", line 59, in main
    stations_to_db(r.text)
  File "/var/folders/7r/5dq81jms6sddg6kh57c8ypwm0000gn/T/ipykernel_56905/1381663942.py", line 10, in stations_to_db
    stations = json.loads(text)
NameError: name 'json' is not defined



NameError: name 'engine' is not defined

In [ ]:
url = "https://api.openweathermap.org/data/2.5/weather?q=Dublin&units=metric&appid=3f8d6a1b1c08e87d9aa40e783517b136"

In [ ]:
def write_to_file2(text):
    now = datetime.datetime.now()
    with open(f"data/weather_{now}".replace(" ", "_"), "w") as f:
        f.write(text) 
        
def weather_to_db(text):
    weather = json.loads(text)
    print(type(weather),len(weather))
    print(text)

    for w in weather:
        print(w)
        vals = (
            weather.get("clouds"),
            weather.get("coord").get("lon"),
            weather.get("coord").get("lan"),
            weather.get("weather").get("main"),
            weather.get("main").get("temp"),
            weather.get("main").get("humidity"),
            weather.get("wind").get("speed"),
            weather.get("wind").get("deg"),
            weather.get("sys").get("sunset"),
            weather.get("sys").get("sunirise")
        )
        print("\n\n\nVALS:{}\n\n\n".format(vals))
        engine.execute("insert into weather values(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)", vals)

    
def weather_main():
    while True:
        try:
            now = datetime.datetime.now()
            r = requests.get(url)
            print(r,now)

            weather_to_db(r.text)
            time.sleep(5 * 60)
        except:
            print(traceback.format_exc())
            if engine is None:
                return

if __name__ == "__weather__main__":
    weather_main() 